In [1]:
#데이터 핸들링 라이브러리
import pandas as pd
import numpy as np
#데이터 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.family':'AppleGothic'})
mpl.rc('axes', unicode_minus=False)
%config InlineBackend.figure_format = 'retina'

## 데이터 전처리 계획
1. 데이터 불러오기, 결측치 처리하기
2. 상대적으로 중요성이 떨어져보이는 피처 삭제, 필요하면 피처명을 직관적으로 다시 네이밍하기
3. 비숫자-문자열 독립변수 인코딩하기
4. 데이터 정규화
5. 훈련-테스트셋 분리

## 모델 학습 계획
1. 모델 생성
2. Cross_validation을 통한 모델학습
3. Accuracy, Precision,Recall, ROC 4개의 지표로 분류모델의 정확도 평가하기

In [2]:
#1. 데이터 불러오기 from TAAS: http://taas.koroad.or.kr/web/shp/sbm/initGisAnals.do?menuId=WEB_KMP_GIS_TAS
## 해당 경로에 있는 .csv 파일명 리스트 가져오기

import os
import time
t_0= time.time()

path = './교통사고_2021/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xls')] ## 파일명 끝이 .xls인 경우

## csv 파일들을 DataFrame으로 불러와서 concat

death_df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_html(path + i)
    data= data[0]
    death_df = pd.concat([death_df,data])

death_df = death_df.reset_index(drop = True)
print(f'데이터 불러오기 및 통합에 걸린 시간은 {str(time.time()-t_0)} 입니다')
death_df

데이터 불러오기 및 통합에 걸린 시간은 29.06381106376648 입니다


,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2021010100100022,2021년 1월 1일 02시,금요일,강원도 양양군 현남면,중상사고,0,1,1,0,차량단독 - 공작물충돌,...,맑음,단일로 - 기타,승용,여,19세,상해없음,NaN,NaN,NaN,NaN
1,2021010100100029,2021년 1월 1일 04시,금요일,강원도 횡성군 둔내면,경상사고,0,0,2,0,차량단독 - 공작물충돌,...,맑음,교차로 - 교차로안,승용,남,21세,경상,NaN,NaN,NaN,NaN
2,2021010100100040,2021년 1월 1일 08시,금요일,강원도 강릉시 강문동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,단일로 - 기타,화물,남,53세,상해없음,보행자,여,20세,중상
3,2021010100100065,2021년 1월 1일 11시,금요일,강원도 강릉시 옥계면,경상사고,0,0,4,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,남,43세,상해없음,화물,남,49세,경상
4,2021010100100089,2021년 1월 1일 13시,금요일,강원도 동해시 천곡동,경상사고,0,0,2,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,62세,상해없음,승용,남,24세,경상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194089,2021123100100335,2021년 12월 31일 16시,금요일,충청남도 천안시 동남구 광덕면,경상사고,0,0,3,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,여,32세,경상,승용,남,42세,경상
194090,2021123100100383,2021년 12월 31일 17시,금요일,충청남도 부여군 장암면,중상사고,0,1,0,0,차대차 - 추돌,...,맑음,단일로 - 기타,승용,남,24세,상해없음,자전거,남,78세,중상
194091,2021123100100384,2021년 12월 31일 17시,금요일,충청남도 천안시 동남구 청당동,경상사고,0,0,3,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,25세,경상,승용,남,22세,경상
194092,2021123100100578,2021년 12월 31일 21시,금요일,충청남도 보령시 명천동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,기타 - 기타,승용,남,59세,상해없음,승용,여,28세,경상


In [3]:
"""
결측치 확인하기 => 피해운전자의 차종,성별,연령,상해정도를 파악하지 못하는 데이터가 6687건이 존재함
원본데이터 194094에 비하면 삭제해도 학습에 영향력을 크게 미치지 못하뿐더러, 이러한 데이터들이
특정 편향성을 갖고 존재할 것이라 보기 어렵다(아마도 기록 누락에 의해 이러한 데이터가 존재할 것이기 때문)
그렇기 때문에 데이터를 제거해도 될것이라 판단하였음
"""
death_df.isnull().sum()
death_df.dropna(inplace=True)
death_df.isnull().sum()

사고번호          0
사고일시          0
요일            0
시군구           0
사고내용          0
사망자수          0
중상자수          0
경상자수          0
부상신고자수        0
사고유형          0
법규위반          0
노면상태          0
기상상태          0
도로형태          0
가해운전자 차종      0
가해운전자 성별      0
가해운전자 연령      0
가해운전자 상해정도    0
피해운전자 차종      0
피해운전자 성별      0
피해운전자 연령      0
피해운전자 상해정도    0
dtype: int64

In [4]:
#데이터 정보 찾기
print(death_df.info())
death_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187407 entries, 2 to 194093
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   사고번호        187407 non-null  int64 
 1   사고일시        187407 non-null  object
 2   요일          187407 non-null  object
 3   시군구         187407 non-null  object
 4   사고내용        187407 non-null  object
 5   사망자수        187407 non-null  int64 
 6   중상자수        187407 non-null  int64 
 7   경상자수        187407 non-null  int64 
 8   부상신고자수      187407 non-null  int64 
 9   사고유형        187407 non-null  object
 10  법규위반        187407 non-null  object
 11  노면상태        187407 non-null  object
 12  기상상태        187407 non-null  object
 13  도로형태        187407 non-null  object
 14  가해운전자 차종    187407 non-null  object
 15  가해운전자 성별    187407 non-null  object
 16  가해운전자 연령    187407 non-null  object
 17  가해운전자 상해정도  187407 non-null  object
 18  피해운전자 차종    187407 non-null  object
 19  피해운전자 성별    187407 non-

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
2,2021010100100040,2021년 1월 1일 08시,금요일,강원도 강릉시 강문동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,단일로 - 기타,화물,남,53세,상해없음,보행자,여,20세,중상
3,2021010100100065,2021년 1월 1일 11시,금요일,강원도 강릉시 옥계면,경상사고,0,0,4,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,남,43세,상해없음,화물,남,49세,경상
4,2021010100100089,2021년 1월 1일 13시,금요일,강원도 동해시 천곡동,경상사고,0,0,2,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,62세,상해없음,승용,남,24세,경상
5,2021010100100123,2021년 1월 1일 15시,금요일,강원도 강릉시 포남동,사망사고,1,0,0,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,원동기,남,16세,상해없음,승용,여,50세,상해없음
6,2021010100100146,2021년 1월 1일 17시,금요일,강원도 강릉시 옥천동,경상사고,0,0,2,0,차대차 - 추돌,...,맑음,교차로 - 교차로부근,승용,여,33세,상해없음,승용,남,27세,경상


In [21]:
death_df['사고내용'].value_counts()

0    136117
2     46659
1      2140
Name: 사고내용, dtype: int64

In [6]:
#2.불필요해보이는 컬럼 제거
'''
사고번호는 각 사고에 대해 일대일 대응되는 key값으로 나중에 데이터를 결합시키는데 사용될 수 있어 보이므로 남겨둠
사망자수,중상자수,경상자수,부상신고자는 '피해운전자 상해정도'에 필요한 정보가 담겨있으므로 삭제한다
위치정보는 사용하지 않으므로 시군구도 삭제한다

'''
death_df.drop(['시군구','사망자수','중상자수','경상자수','부상신고자수','노면상태','가해운전자 상해정도','피해운전자 상해정도'],axis=1,inplace=True)

## 컬럼 핸들링하기
##시간에 따른 시계가 교통사고의 요인중 하나일 수 있으므로 발생년월일시에서 시간만 떼어 새로운 컬럼으로 만듬
time_lst= list(death_df['사고일시'])
hour_lst=[]
for time in time_lst:
    hour_lst.append((str(time)[-3:-1]))
death_df['사고시각']= hour_lst
#사고일시 컬럼은 삭제
death_df.drop(['사고일시'],axis=1,inplace=True)


In [7]:


#피해운전자, 가해운전자 연령 미분류 삭제하기
idx= death_df[death_df['피해운전자 연령']=='미분류'].index
death_df.drop(idx,inplace=True)
idx= death_df[death_df['피해운전자 연령']=='98세 이상'].index
death_df.drop(idx,inplace=True)

idx2= death_df[death_df['가해운전자 연령']=='미분류'].index
death_df.drop(idx2,inplace=True)


#나이 컬럼: 숫자단위만 뽑기
suspect_lst= list(death_df['가해운전자 연령'])
suspect_old= []
for old in suspect_lst:
    suspect_old.append((old)[:-1])

death_df['가해운전자 연령']=suspect_old

victim_lst= list(death_df['피해운전자 연령'])
victim_old= []
for old in victim_lst:
    victim_old.append((old)[:-1])
death_df['피해운전자 연령']=victim_old


In [8]:
death_df=death_df.reindex(columns=['사고시각','사고유형','법규위반','기상상태','도로형태','가해운전자 차종','가해운전자 성별','가해운전자 연령','피해운전자 차종'\
                           ,'피해운전자 성별','피해운전자 연령','사고번호','사고내용'])

In [9]:
#데이터 자료형 변환: 문자열-> 정수형
death_df['사고시각']=death_df['사고시각'].astype('int')
death_df['가해운전자 연령']=death_df['가해운전자 연령'].astype('int')
death_df['피해운전자 연령']=death_df['피해운전자 연령'].astype('int')

![](https://scikit-learn.org/stable/_images/multi_org_chart.png)

In [10]:
?#우선 의사결정나무와 랜덤포레스트 머신러닝 기법을 활용해보도록 한다. 그러므로 범주형 변수의 인코딩으로 레이블인코딩을 해도 되게 된다
#3.범주형 변수 인코딩하기
#한번만 인코딩해야됨. 두번하면 안됨
from sklearn.preprocessing import LabelEncoder

categories_backup=[]
features=['사고유형','법규위반','기상상태','도로형태','가해운전자 차종','가해운전자 성별','피해운전자 차종','피해운전자 성별','사고내용']
for feature in features:
    LE= LabelEncoder()
    LE= LE.fit( death_df[feature])
    #카테고리 백업해두기
    categories_backup.append(list(LE.classes_))
    #인코딩하기
    death_df[feature]= LE.transform(death_df[feature])

Object `#우선 의사결정나무와 랜덤포레스트 머신러닝 기법을 활용해보도록 한다. 그러므로 범주형 변수의 인코딩으로 레이블인코딩을 해도 되게 된다` not found.


In [11]:
#백업해둔 카테고리명 다시 한번 확인하기
categories_backup

[['차대사람 - 기타',
  '차대사람 - 길가장자리구역통행중',
  '차대사람 - 보도통행중',
  '차대사람 - 차도통행중',
  '차대사람 - 횡단중',
  '차대차 - 기타',
  '차대차 - 정면충돌',
  '차대차 - 추돌',
  '차대차 - 측면충돌',
  '차대차 - 후진중충돌',
  '철길건널목 - 철길건널목'],
 ['과속',
  '교차로운행방법위반',
  '기타',
  '보행자보호의무위반',
  '불법유턴',
  '신호위반',
  '안전거리미확보',
  '안전운전불이행',
  '중앙선침범',
  '직진우회전진행방해',
  '차로위반'],
 ['기타', '눈', '맑음', '비', '안개', '흐림'],
 ['교차로 - 교차로부근',
  '교차로 - 교차로안',
  '교차로 - 교차로횡단보도내',
  '기타 - 기타',
  '단일로 - 고가도로위',
  '단일로 - 교량',
  '단일로 - 기타',
  '단일로 - 지하차도(도로)내',
  '단일로 - 철길건널목',
  '단일로 - 터널',
  '미분류 - 미분류',
  '주차장 - 주차장'],
 ['개인형이동수단(PM)',
  '건설기계',
  '기타불명',
  '농기계',
  '사륜오토바이(ATV)',
  '승용',
  '승합',
  '원동기',
  '이륜',
  '자전거',
  '특수',
  '화물'],
 ['기타불명', '남', '여'],
 ['개인형이동수단(PM)',
  '건설기계',
  '기타불명',
  '농기계',
  '보행자',
  '사륜오토바이(ATV)',
  '승용',
  '승합',
  '원동기',
  '이륜',
  '자전거',
  '특수',
  '화물'],
 ['남', '여'],
 ['경상사고', '사망사고', '중상사고']]

In [12]:
#독립변수와 종속변수로 분리하기
y= death_df['사고내용']
X= death_df.drop(['사고번호','사고내용'],axis=1)

#데이터 분할하기
from sklearn.model_selection import train_test_split
#classification용 모델 라이브러리 가져오기
from lightgbm import LGBMClassifier

#전체 데이터중 70퍼센트를 학습에 30퍼센트를 테스트 데이터로 분리한다
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

#위의 학습 데이터중 90퍼센트를 학습 데이터에, 10퍼센트는 검증용 데이터로 분리한다
X_tr,X_val,y_tr,y_val= train_test_split(X_train,y_train,test_size=0.1)

#n_estimators는 동일하게 400을 설정한다
lgbm_wrapper= LGBMClassifier(n_estimators=400,learning_rate=0.05)

#lightLGBM은 조기 중단 수행이 가능하다
evals= [(X_tr,y_tr),(X_val,y_val)]
lgbm_wrapper.fit(X_tr,y_tr,early_stopping_rounds=50,eval_metric='logloss',eval_set=evals,verbose=True)
y_pred= lgbm_wrapper.predict(X_test)

pred_proba= lgbm_wrapper.predict_proba(X_test)[:,1]

/Users/wooseongkyun/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/wooseongkyun/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's multi_logloss: 0.615762	valid_1's multi_logloss: 0.614823
[2]	training's multi_logloss: 0.608184	valid_1's multi_logloss: 0.607441
[3]	training's multi_logloss: 0.601774	valid_1's multi_logloss: 0.601169
[4]	training's multi_logloss: 0.596288	valid_1's multi_logloss: 0.595765
[5]	training's multi_logloss: 0.591522	valid_1's multi_logloss: 0.591055
[6]	training's multi_logloss: 0.587297	valid_1's multi_logloss: 0.586987
[7]	training's multi_logloss: 0.583621	valid_1's multi_logloss: 0.583442
[8]	training's multi_logloss: 0.580318	valid_1's multi_logloss: 0.580237
[9]	training's multi_logloss: 0.577392	valid_1's multi_logloss: 0.577518
[10]	training's multi_logloss: 0.574773	valid_1's multi_logloss: 0.57506
[11]	training's multi_logloss: 0.572438	valid_1's multi_logloss: 0.572858
[12]	training's multi_logloss: 0.570307	valid_1's multi_logloss: 0.570835
[13]	training's multi_logloss: 0.568379	valid_1's multi_logloss: 0.568998
[14]	training's multi_logloss: 0.566632	valid_1'

In [13]:
y

2         2
3         0
4         0
5         1
6         0
         ..
194089    0
194090    2
194091    0
194092    0
194093    2
Name: 사고내용, Length: 184916, dtype: int64

In [14]:
#하이퍼 파라미터 튜닝하기
from hyperopt import hp
lgbm_search_space={'num_leaves':hp.quniform('num_leaves',32,64,1),
                   'max_depth': hp.quniform('max_depth',100,160,1),
                   'min_child_samples':hp.quniform('min_child_samples',60,100,1),
                   'subsample':hp.uniform('subsample',0.7,1),
                   'learning_rate':hp.uniform('learning_rate',0.01,0.2)
                   }


In [15]:
from sklearn.model_selection import KFold
from hyperopt import fmin,tpe,Trials
import random

def objective_fun(search_space):
    lgbm_clf= LGBMClassifier(
                             class_weight='balanced',
                             n_estimators=100,
                             num_leaves=int(search_space['num_leaves']),
                             max_depth=int(search_space['max_depth']),
                             min_child_samples=int(search_space['min_child_samples']),
                             subsample= search_space['subsample'],
                             learning_rate=search_space['learning_rate']
                             )
    #3개의 k-fold 방식으로 평가된 rou_auc를 담는 list
    roc_auc_lst =[]

    #3개의 k-fold 방식을 적용하기
    kf= KFold(n_splits=3)
    #X_train을 다시 학습과 검증용 데이터로 분리한다
    for tr_idx, val_idx in kf.split(X_train):
        #kf.split(X_train)으로 추출된 학습과 검증 idx값으로 학습과 검증 데이터 셋 분리
        X_tr,y_tr= X_train.iloc[tr_idx],y_train.iloc[tr_idx]
        X_val,y_val=X_train.iloc[val_idx], y_train.iloc[val_idx]

        #early stopping은 100회로 설정하고 추출된 학습과 검증 데이터로 lgbm_classfier를 수행한다
        lgbm_clf.fit(X_tr,y_tr,early_stopping_rounds=30,eval_metric='auc',eval_set=[(X_tr,y_tr),(X_val,y_val)])

        #1로 예측한 확률값 추출 후 roc_auc를 계산하고 평균 roc_auc를 계산하기 위해 list에 결과값을 담는다
        score= roc_auc_score(y_val,lgbm_clf.predict_proba(X_val)[:,1])
        roc_auc_lst.append(score)

    #3개의 K-fold값으로 계산된 roc_auc값의 평균값을 반환하되
    #Hyperopt는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환한다
    return -1 * np.mean(roc_auc_lst)


In [16]:
trials= Trials()
#fmin() 함수를 호출, max_evals를 지정된 횟수만큼 반복 후 목적함수의 최솟값을 찾는  최적입력값을 추출한다
best= fmin(fn=objective_fun,space=lgbm_search_space,algo=tpe.suggest,
           trials=trials,rstate=np.random.default_rng(seed=50),max_evals=50)



  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/wooseongkyun/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

[LightGBM] [Fatal] Multiclass objective and metrics don't match
job exception: Multiclass objective and metrics don't match



  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


LightGBMError: Multiclass objective and metrics don't match

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,roc_auc_score


print('오차 행렬 값은: \n',confusion_matrix(y_test,y_pred))
print(f'accuracy 스코어는 {accuracy_score(y_test,y_pred):.5f} 입니다')
print(f"precision 스코어는 {precision_score(y_test,y_pred,average='macro'):.5f}")
print(f"recall 스코어는 {recall_score(y_test,y_pred,average='macro'):.5f}")
